## Scraping: Άρθρα από το Institute of Economic Affairs (https://iea.org.uk/blog)


### Πάμε να πάρουμε περιεχόμενο από την αρχική σελίδα του Institute of Economic Affairs. 

Θέλουμε:

Τίτλους (Headlines)

Συνόψεις (Summary)

Συγγραφείς (Author)

Υπερσυνδέσεις (link)

Ημερομηνία δημοσίευσης (date)

Ξεκινώντας πρέπει να εισάγουμε τα απαραίτητα libraries.

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
import ast
import time

### Ορίζουμε τα headers για να νομίζει ο σέρβερ της ιστοσελίδας ότι πρόκειται για browser και όχι για εμάς!

In [ ]:
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.77 Safari/537.36"}

### Παίρνουμε την σελίδα και την μετατρέπουμε σε ένα αντικείμενο beautiful soup (BS object).

In [ ]:
response = requests.get("https://iea.org.uk/blog", headers=headers)
#response = requests.get("https://www.nytimes.com/", headers=headers)
doc = BeautifulSoup(response.text)

In [ ]:
doc

### Πάμε στην ιστοσελίδα και ελέγχουμε τι ψάχνουμε να βρούμε, για παράδειγμα εδώ θέλουμε το h2 και συγκεκριμένα όλα τα a (δηλαδή τα a href)

In [ ]:
# Τραβάμε τα links και τα σώζουμε σε μια λίστα
links = doc.select("h2 a") 

In [ ]:
links

### είναι διπλά και εμείς θέλουμε τα μονά 

In [ ]:
links = set(links)
print(links)

### Μπορούμε να φιλτράρουμε σύμφωνα με λέξεις, αυτό έχει νόημα όταν ψάχνουμε κάποια συγκεκριμένη θεματολογία. Ας φιλτράρουμε π.χ. για economic.


In [ ]:
filtered = [link for link in links if 'economic' in link.text.lower()]
filtered

### Loops στα Pandas!

Για να κρατήσουμε μόνο το ωφέλιμο λινκ χωρίς την html χρησιμοποιούμε ένα loop μέσα στα [ ]. 

Αυτό ονομάζεται *list comprehension*.

In [ ]:
urls = [link['href'] for link in links]
urls

### Βλέπουμε ότι οι τίτλοι εμφανίζονται μέσα στο λινκ. 
Ας τους τραβήξουμε με τον τρόπο που μάθαμε!

In [ ]:
titles = [link.text for link in links]
titles

## Φτιάχνουμε το function για το scraping που θα το χρησιμοποιήσουμε για να πάρουμε την λίστα με τα λινκς των άρθρων!

In [ ]:
# Είσοδος: μια λίστα (base list) με τα links που έχουν τα URLs που θα συμπεριληφθούν στην λίστα για scraping
# Έξοδος: λίστα με τα scraped links
#Θα μπορούσα να είχα φτιάξει μια λίστα με λέξεις accepted_words, ώστε να τσέκαρα μόνο για συγκεκριμένα keywords μέσα στα links
def get_scraped_links(base_list): #, accepted_words):
    # Ανοίγουμε μια άδεια λίστα μέσα στην οποία στην συνέχεια θα μπουν τα scraped links
    scraped_links = []
    
    # Ελέχγει ένα - ένα όλα τα URLs μέσα στην base list για να τραβήξειόλα τα links που βρίσκονται μέσα στα αρχικά links 
    # Προσθέτει τα νέα links στην λίστα με τα scraped links
    for url in base_list:
        parent = 'https://' + url.split('/')[2]
        req = requests.get(url, headers)
        print(req)
        soup = BeautifulSoup(req.content, 'html.parser')
        
        # Για όλα τα links στα οποία αναφέρεται το link που βρίσκεται μέσα στο base_list:
        for link in soup.select("h2 a"):
            
            # Τράβα το url. Αν πρόκειται για εσωτερικό link που δεν ξεκινά με το domain, φτιάχνει το full URL ώστε να το κάνουμε scrape μετά
            try:
                l = link.get('href')
                if l[0] == '/': 
                    l = parent + l
                    print(l)
                    scraped_links.append(l)
            
                # Τσεκάρει ότι μέσα στο URL υπάρχει έστω μια λέξη από τη λίστα accepted words
                #if any(w in l for w in accepted_words):
                    #scraped_links.append(l)
            except:
                print('no url')

    # πετάει τα διπλά 
    scraped_links = list(set(scraped_links))
    
    # Τυπώνει το νούμερο των links που είναι scraped
    print(len(scraped_links))
    
    return scraped_links

## Πάμε να φτιάξουμε μια λούπα για να πάρουμε πρώτα τα links των επιμέρους σελίδων του website. 
Παρατηρούμε ότι υπάρχουν 343 σελίδες, άρα...

In [ ]:
num = 1
pages = []
for i in range(0,342):
    num = num + 1
    #print(num)
    page = 'https://iea.org.uk/blog/'
    newpage = page + '?fwp_paged='+ str(num)
    pages.append(newpage)
    print(newpage)

In [ ]:
#έλεγχος
pages[-1]

In [ ]:
urls = get_scraped_links(pages) #, accepted_words)

In [ ]:
### Φτιάχνω ένα function για να πάρω το κάθε λινκ

In [ ]:
from bs4 import BeautifulSoup
import requests
import time

headers = {'user-agent': 'my-app/0.0.1'}
PAUSE_TIME = 3.5

def get_text(url):
    r = requests.get(url, headers = headers)
    time.sleep(PAUSE_TIME)
    s = BeautifulSoup(r.content, 'html.parser')
    link = s.find("h2 a")
    
    return([s, link])

### Κάνω scraping κάθε σελίδα για να μαζέψω όλα τα links των άρθρων

In [ ]:
scraped_links = []
titles = []
counter = 0
base_url = "https://iea.org.uk/blog/?fwp_paged="
for i in range(0,342):
    url = base_url + str(i)
    #print(url)
    page = requests.get(url, headers = headers)
    s = BeautifulSoup(page.content, 'html.parser')
    #print(s)
    links = s.select("h2 a")
    links = set(links)
    
    url = [link['href'] for link in links]
    
    #title = [link.text for link in links]
    
    
    print(url)

    for l in url:
        link = l
        print(l)
        scraped_links.append(l)


    
    # Τυπώνει το νούμερο των links που είναι scraped
print(len(scraped_links)) 

scraped_links = (set(scraped_links))

In [ ]:
scraped_links

In [ ]:
len(scraped_links)

### Αφού μάζεψα τα links των άρθρων τα σώζω για να μην τα χάσω και στην συνέχεια θα γράψω κώδικα που θα παίρνει τα στοιχεία του κάθε άρθρου ξεχωριστά!

In [ ]:
import pandas as pd
df = pd.DataFrame(scraped_links, columns= ['url'])

In [ ]:
df.to_csv("iea.csv", index=False)

In [ ]:
df = pd.read_csv('iea.csv')

In [ ]:
df

In [ ]:
urls = df['url'].tolist()
urls

In [ ]:
len(urls)

# Πάμε να κάνουμε scrape την κάθε σελίδα που περιέχει ένα άρθρο


In [ ]:
response = requests.get("https://iea.org.uk/the-low-hanging-fruit-on-the-brexit-tree-repeal-the-tobacco-products-directive/", headers=headers)

doc = BeautifulSoup(response.text)

In [ ]:
# Τίτλος
titles = doc.select("h2")[0].text
titles

In [ ]:
#Συγγραφέας
author = doc.select("div.ph-header-block div")[1].text.strip()
author


In [ ]:
#Ημερομηνία
date = doc.select("div.ph-header-block div")[2].text.strip()
date

In [ ]:
#Κείμενο
article = doc.select("#modal-ready")[0].text
article

### Πάμε να κάνουμε το ίδιο για όλα τα urls

In [ ]:
urls

In [ ]:
from bs4 import BeautifulSoup
import os
import requests
from datetime import datetime
import ast
import time


#def remove_non_ascii(text):
#    return ''.join([i if ord(i) < 128 else ' ' for i in text])

list=[]
for i in urls[0:15]:  #βάζω συγκεκριμένο αριθμό για τεστ
    dic = {}
    #url = base_url + str(i)
    print(i)
    page = requests.get(i, headers = headers)
    soup = BeautifulSoup(page.content, 'html.parser')

    
    title = soup.select("h2")

    if (title):
        dic['title'] = soup.select("title")[0].text
    else:
        print("ARTICLE NO: ", link, " HAS NO TITLE")
        dic['title'] = "NO TITLE"
    
        
    date = soup.select("div.ph-header-block div")
    if (date):
        dic['date'] = soup.select("div.ph-header-block div")[2].text.strip()
    else:
        print("ARTICLE NO: ", link, " HAS NO date")
        dic['date'] = "NO date"
        
        
    author = soup.select("div.ph-header-block div")
    if (author):
        dic['author'] = soup.select("div.ph-header-block div")[1].text.strip()
    else:
        print("ARTICLE NO: ", link, " HAS NO author")
        dic['author'] = "NO author"        
        
    
    article =  soup.select("#modal-ready")
    if (article):
        dic['body'] = soup.select("#modal-ready")[0].text
    else:
        print("ARTICLE NO: ", link, " HAS NO body")
        dic['body'] = "NO body"  
    #article =  soup.select("#modal-ready")[0].text
    
    #article_text = [a.text for a in article]
    #dic['body'] = article
    
    dic['url'] = i

    
    list.append(dic)
    #print(dic)
    
    time.sleep(0.5) #ορίζω πόση ώρα θα περιμένει μέχρι να ξαναχτυπήσει τον σέρβερ για να πάρει το επόμενο
    

## Κάνω την λίστα dataframe

In [ ]:
df = pd.DataFrame(list)
df

## Πετάω τυχον διπλά

In [ ]:
df.drop_duplicates(subset=['url'], keep='first', inplace = True)
df.reset_index(drop=True, inplace = True)

In [ ]:
df.shape

## Και σώζω το dataframe!

In [ ]:
df.to_csv("iea_articles.csv", index = False)
df = pd.read_csv("iea_articles.csv")
df


## Κάνω τις ημερομηνίες τύπο datetime

In [ ]:
from datetime import timedelta
from datetime import datetime as dt

df['date'] = pd.to_datetime(df['date'], errors='coerce')
df

### Φτιάχνω ένα νέο column για τον χρόνο

In [ ]:
df['year'] = df['date'].dt.strftime('%Y')

### Σορτάρω

In [ ]:
df.sort_values(by=['date'], inplace=True, ascending=False)

### Κρατάω μόνο όσα άρθρα έχουν κείμενο και ημερομηνία.

In [ ]:
df[df['body'].notna()]

In [ ]:
df[df['date'].notna()]

### Ξανασώζω το καθαρό df

In [ ]:
df.to_csv("iea_articles.csv", index=False)